# Importing necessary Modules

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.externals import joblib
from normalization_coefficiants import * # this file contains the normalization coefficiants that were used on the dataset

C:\Program Files\Python36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Below we specify the models inputs and target predictions

In [2]:
X_cols = ['Cement (%W)', 'Water (%W)', 'Fine Agg (%W)', 'PCMs (%W)',
          'Fly ash (%W)', 'Silica Fume (%W)', 'Cenposphere (%W)', 'BFS (%W)',
          'Coarse Agg (%W)', 'AC (%W)', 'Superplasticiser (%W)',
          'Fiber Amount (%W)', 'PVA', 'PE', 'STEEL', 'CaCO3',
          'Polypropylene', 'Carbon', 'Glass', 'Basalt', 'UHMWPE',
          'F. Diameter (Micro-Meter)', 'Fiber Length (mm)',
          'Fiber Tensile Strength (Mpa)', 'Fiber Elastic Modulus (Gpa)',
          'Rubber', 'Lightw Agg',
          'MC (Viscosity Agent)', 'AEA', 'Oilling Agent/coating', 'HFV-Casting',
          'Graphene Oxide', 'Defoamer', 'Temperature', 'Water Curing',
          'Air Curing']

NX_cols = ['Fiber Amount (%W)', 'PVA', 'PE', 'STEEL', 'CaCO3',
           'Polypropylene', 'Carbon', 'Glass', 'Basalt', 'UHMWPE',
           'F. Diameter (Micro-Meter)', 'Fiber Length (mm)',
           'Fiber Tensile Strength (Mpa)', 'Fiber Elastic Modulus (Gpa)',
           'cs', 'ux2', 'ux1', 'us1', 'fs2', 'fx1', 'fs1', 'fx2', 'us2']

y_cols = ['cs', 'ux2', 'ux1', 'us1', 'fs2', 'fx1',
          'fs1', 'fx2', 'us2']  # Modified for Information Gain
full_feed = y_cols

The Model uses sequential stacking technique to make its predictions.
We first start by loading the random forest regressors, then we load the neural network models

In [3]:
models = []

for i, model_name in enumerate(y_cols):
    feed = y_cols[0:i][:]

    model = {"name": model_name,
             "X": X_cols,
             "y": [model_name],
             "feed": feed}
    models.append(model)

rf = {}
fN = {}

for model in models:
    name = model['name']

    rf[name] = joblib.load(f"Models/FRST/rf-{name}.pkl")
    fN[name] = tf.keras.models.load_model(f'Models/FDNN Ensemble/N-{name}.h5')

W0224 14:51:24.698518 24532 deprecation.py:506] From C:\Program Files\Python36\lib\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0224 14:51:24.700518 24532 deprecation.py:506] From C:\Program Files\Python36\lib\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0224 14:51:24.733521 24532 deprecation.py:506] From C:\Program Files\Python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops

# Encoding and Decoding Functions
The FDNN models are trained using normalized data, therefore it is important that we normalize the data using min max normalization. The coefficiants for the inputs are stored in a separate file named "normalization_coefficiants".
The predictions made using the $FDNN(X_s)$ are also normalized, therfore we use the decode function to convert them into the correct format.

In [ ]:
def encode(df):
    encoded = []

    for name, value in zip(df.columns, df.values.T):
        value = float(value[0])
        if name in y_cols:
            V = y_decodes[name]
        elif name in X_cols:
            V = X_decodes[name]

        val = (value - V["min"]) / (V["max"] - V["min"])  # normalization
        #val = (value - V["mean"]) / V["std"]
        encoded.append(val)
    V_en = pd.DataFrame(np.array(encoded), index=df.columns)
    return V_en.T

In [ ]:
def decode(df):
    decoded = []

    for name, value in zip(df.columns, df.values.T):
        value = float(value[0])
        if name in y_cols:
            V = y_decodes[name]
        elif name in X_cols:
            V = X_decodes[name]

        # val = (value * V["std"]) + V["mean"]  # standardiztion
        val = value * (V["max"] - V["min"]) + V["min"]  # normalization
        decoded.append(val)
    V_de = pd.DataFrame(np.array(decoded), index=df.columns)
    return V_de.T

# Prediction functions
Making predictions using the ensemble model takes in regard that sequentail stacking needs to be performed, and since we are using sklearn and tensorflow, the prediction function is customized as seen below

In [ ]:
def nan_predict(predictions):
    array = []
    for i in range(len(predictions)):
        array.append(np.nan)
    df = pd.DataFrame(np.array(array))
    return df


def get_predictions(predictions, column_names):
    array = []
    for i in range(len(predictions)):
        array.append(predictions[i])
    df = pd.DataFrame(np.array(array), columns=column_names)
    return df


def ensemble_predict(x):
    list_of_xs = []
    for value in x.values():
        list_of_xs.append(value)

    xs = np.array(list_of_xs).astype(np.float).reshape(1, -1)
    xs = pd.DataFrame(xs, columns=X_cols)
    xs = encode(xs)

    y_pred = {}
    for output in y_cols:
        y_pred[output] = output

    predictions = []

    x_i = xs.copy()

    for model in models:
        name = model['name']
        feed = full_feed

        X_data = x_i.copy()

        for feed_name in feed:
            rf_predictions = rf[feed_name].predict(X_data)
            rf_predictions = pd.DataFrame(rf_predictions, columns=[feed_name])
            X_data = pd.concat([X_data, rf_predictions], axis=1)

        X_data = X_data[y_cols]
        fN_preds = fN[name].predict(X_data)

        predictions.append(get_predictions(
            fN_preds, [y_pred[name]]).values[0][0])

    predictions = np.array(predictions).reshape(1, 9)
    predictions = pd.DataFrame(predictions)
    predictions.columns = y_cols
    predictions = decode(predictions).values
    predictions = predictions.flatten().tolist()

    return predictions

# Example prediction:
We show how our FDNN model can be used to make a prediction for an SHCC mix design

the predictions and true values for the properties are:
* Compressive strength: prediction = 15.252 MPa, True value = 13.90 MPa
* Peak tensile stress (ux2): prediction = 3.637 MPa, True value = 3.97 Mpa
* Peak tensile strain (us2): prediction = 2.893 %, True value = 3.10 %

and the prediction are shown after running the cell below

In [ ]:
# Mix Design Inputs
x = {}
exit_app = False

while exit_app != True:
    print("\n\nPlease Enter the Following Parameters: \n")

    for name in X_cols:
        x[name] = float(input(f"{name} = "))

    print("\nPlease Wait for predictions...\n")
    mechanical_properties = ['Compressive strength (cs)',
                            'peak tensile stress (ux2)',
                            'first crack tensile stress (ux1)',
                            'first crack tensile strain (us1)',
                            'peak flexural strain (fs2)',
                            'first crack flexural Strength (fx1)',
                            'first crack flexural strain (fs1)',
                            'peak flexural strength (fx2)',
                            'peak tensile strain (us2)']
    results = ensemble_predict(x)

    for mp_name, p in zip(mechanical_properties, results):
        print(f"{mp_name} = {p}")

    exit_response = input("Press 'R' to make another prediction or another key to exit")
    if exit_response.lower() == 'r':
        exit_app = True



Please Enter the Following Parameters: 



Cement (%W) =  0.1128
Water (%W) =  0.158
Fine Agg (%W) =  0.219
PCMs (%W) =  0
Fly ash (%W) =  0.5
Silica Fume (%W) =  0
Cenposphere (%W) =  0
BFS (%W) =  0
Coarse Agg (%W) =  0
AC (%W) =  0
Superplasticiser (%W) =  0.0021
Fiber Amount (%W) =  0.0126
PVA =  0.02
PE =  0
STEEL =  0
CaCO3 =  0
Polypropylene =  0
Carbon =  0
Glass =  0
Basalt =  0
UHMWPE =  0
F. Diameter (Micro-Meter) =  39
Fiber Length (mm) =  12
Fiber Tensile Strength (Mpa) =  1600
Fiber Elastic Modulus (Gpa) =  42
Rubber =  0
Lightw Agg =  0
MC (Viscosity Agent) =  0
AEA =  0
Oilling Agent/coating =  1
HFV-Casting =  0
Graphene Oxide =  0
Defoamer =  0
Temperature =  200
Water Curing =  1
Air Curing =  27



Please Wait for predictions...

Compressive strength (cs) = 15.252283689379691
peak tensile stress (ux2) = 3.637191435694694
first crack tensile stress (ux1) = 2.5681637763977054
first crack tensile strain (us1) = 0.0584994046189785
peak flexural strain (fs2) = 4.25364328622818
first crack flexural Strength (fx1) = 9.465974307060241
first crack flexural strain (fs1) = 0.6121749324457987
 peak flexural strength (fx2) = 7.552531075477601
peak tensile strain (us2) = 2.8930210228562356
